In [1]:
%run qae_utils.py
from qae_utils import *
from qae_utils import state_preparation_ansatz
from typing import Iterable, Optional, Union
from scipy.optimize import minimize
from qiskit import *
from qiskit import execute

In [12]:
global verbose
verbose = True

#### #Phase0

In [15]:
qpe(0.25), qpe(0.25, trainable= True, p_param= 0.25)

NameError: name 'execute' is not defined

In [6]:
p_guess = 0.24
p= 0.25

r = minimize(to_minimize, p_guess, method= 'Nelder-Mead')

inside-> to_minimize
inside-> qpe
 'p'=   0.2
 'p_param'=  0.24
ourside-> qpe, p_est 0.23110780748033713
cost  [7.90710878e-05]
inside-> to_minimize
inside-> qpe
 'p'=   0.2
 'p_param'=  0.252
ourside-> qpe, p_est 0.25196112424756056
cost  [1.51132413e-09]
inside-> to_minimize
inside-> qpe
 'p'=   0.2
 'p_param'=  0.264
ourside-> qpe, p_est 0.27268432216212385
cost  [7.54174514e-05]
inside-> to_minimize
inside-> qpe
 'p'=   0.2
 'p_param'=  0.258
ourside-> qpe, p_est 0.26361983359248137
cost  [3.15825296e-05]
inside-> to_minimize
inside-> qpe
 'p'=   0.2
 'p_param'=  0.246
ourside-> qpe, p_est 0.23831954450323065
cost  [5.89893966e-05]
inside-> to_minimize
inside-> qpe
 'p'=   0.2
 'p_param'=  0.255
ourside-> qpe, p_est 0.2582073541804409
cost  [1.02871208e-05]
inside-> to_minimize
inside-> qpe
 'p'=   0.2
 'p_param'=  0.249
ourside-> qpe, p_est 0.25029731977205455
cost  [1.68303859e-06]
inside-> to_minimize
inside-> qpe
 'p'=   0.2
 'p_param'=  0.2505
ourside-> qpe, p_est 0.2492358577

In [7]:
qpe(0.25), qpe(0.25, trainable= True, p_param= r.x[0])

inside-> qpe
 'p'=   0.25
inside-> qpe
 'p'=   0.25
 'p_param'=  0.252


(0.248324447925864, 0.2559669994488036)

In [26]:
p_est

0.2511381850103768

#### #Phase1
Prepare PQC to assist the state perparation circuit to be used inside the Grover Operator, $ \mathcal{\hat{Q}_{\beta}} $, where $\beta$ in the set of trainable paramters. (** For single qubit states only; oracle $\mathcal{\hat{O}_{\ket{1}}} = \hat{Z}$ and $\mathcal{\hat{O}_{\ket{0}}} = -\hat{Z}$ ).

$$ \mathcal{\hat{Q}_{\beta}} =  (\mathbb{\hat{I}} - 2 \ket{\tilde{\psi_0}(\beta)} \bra{\tilde{\psi_0}(\beta)} ) \: \mathcal{\hat{O}}
$$

Here the $\ket{\tilde{\psi_0}(\beta)}$ is approximated initial state and is prepard using the PQC as 
$$      \ket{\tilde{\psi_0}(\beta)}\ket{\Gamma_{\beta}} = \mathcal{\hat{P}_{\beta}} ( \ket{0} \ket{\Gamma} )
$$
where $\Gamma$ denotes the ancilla qubits, and $\mathcal{\hat{P}_{\beta}}$ denotes the paramterised quantum circuit.

In [5]:
num_ancillas = 5

params_state = np.random.uniform(low=0, high= 2*pi, size= 2)
params_ancilla= np.random.uniform(low=0, high= 2*pi, size= 2*num_ancillas )

qreg = QuantumRegister(1, name= 'q')
ancillas = AncillaRegister(num_ancillas, name= 'ancilla')
qc = QuantumCircuit(qreg, ancillas)

## set_ansatz ~
qc.ry(params_state[0], qreg)
qc.cx(qreg, ancillas[0])

for index, ancilla in list(enumerate(ancillas))  :
    qc.ry(params_ancilla[index], ancilla)
    qc.cx(ancillas[index], ancillas[(index+1)%(num_ancillas)])
qc.barrier()
for index, ancilla in list(reversed(list(enumerate(ancillas))))  :
    qc.ry(params_ancilla[index], ancilla)
    qc.cx(ancillas[index], ancillas[(index-1)%(num_ancillas)])

qc.cx( ancillas[0], qreg)
qc.ry(params_state[1], qreg)


In [3]:
pqc = state_preparation_ansatz(5, name= 'a', insert_barrier= True)
pqc.initialize(params= 'random')
# pqc.draw(output='mpl')

params = np.random.uniform(low= 0, high= 2 * pi, size= pqc.num_params)

array([2.67784083, 4.75015738, 3.1077421 , 4.46347088, 3.92485517,
       0.84471368, 3.89729513, 0.28942448, 0.57022011, 2.01124589,
       6.14378945, 0.62666572, 0.87258928, 5.0530281 , 1.51556572,
       1.47831417, 4.59569722, 1.29013619, 1.00406628, 1.40327995,
       1.10410359, 1.05707366])

In [ ]:
gr_op = GroverOperator(oracle, state_preparation= state_preparation_circuit, insert_barriers= True)